In [22]:
import json
import random
import re

import pandas as pd

In [3]:
with open("../results/gpt-3.5-turbo-0125/3shots/atomic_2020_duration_estimation.jsonl") as f:
    data = [json.loads(line) for line in f]

In [4]:
keywords = ["minutes", "mins", "hour", "hrs", "1 day", "one day", "a day"]
durations = []
invalid_durations = []
for d in data:
    match = re.search(r'\nAnswer: (.+?)$', d["generated"])
    if match:
        duration = match.group(1)
        if any(keyword in duration for keyword in keywords):
            durations.append(
                {
                    "narrative": d["user_prompt"].split("Narrative:")[1].strip(),
                    "question": d["generated"].split("\n")[0].replace("Question:", "").strip(),
                    "answer": duration
                }
            )
        else:
            invalid_durations.append(
                {
                    "narrative": d["user_prompt"].split("Narrative:")[1].strip(),
                    "question": d["generated"].split("\n")[0].replace("Question:", "").strip(),
                    "answer": duration
                }
            )

In [5]:
print("Valid durations:", len(durations))
print("Invalid durations:", len(invalid_durations))

Valid durations: 57049
Invalid durations: 37920


In [6]:
durations[:10]

[{'narrative': "alcohol causes brewer's droop",
  'question': "How long does it take for brewer's droop to occur after consuming alcohol?",
  'answer': 'It can occur within a few hours after consuming a significant amount of alcohol.'},
 {'narrative': 'anxiety causes sleep loss',
  'question': 'How long does the sleep loss typically last due to anxiety?',
  'answer': 'hours to days'},
 {'narrative': 'arousal causes ejaculation',
  'question': 'How long does it typically take from arousal to ejaculation?',
  'answer': 'a few minutes'},
 {'narrative': 'arson causes sometimes death',
  'question': 'How long does it typically take for firefighters to extinguish a fire caused by arson?',
  'answer': '3 hours'},
 {'narrative': 'bag causes travel',
  'question': 'How long does it typically take to pack for a trip?',
  'answer': '30 minutes to 1 hour'},
 {'narrative': 'beer causes drunkenness',
  'question': 'How long does it typically take for beer to cause drunkenness?',
  'answer': 'It typi

In [7]:
with open("../resources/atomic_2020_narratives.jsonl", "w") as f:
    for d in durations:
        f.write(json.dumps(d) + "\n")

In [15]:
with open("../resources/atomic_2020_narratives.jsonl") as f:
    data = [json.loads(line) for line in f]

In [19]:
sampled = {
    " causes ": [],
    " includes ": [],
    " can be hindered by ": [],
    " happens after ": [],
    " happens before ": [],
    " because ": []
}

In [20]:
for d in data:
    for k in sampled.keys():
        if k in d["narrative"]:
            sampled[k].append(d)
            break

In [21]:
for k, v in sampled.items():
    print(k, len(v))

 causes  173
 includes  1211
 can be hindered by  24247
 happens after  14100
 happens before  14260
 because  102


In [23]:
sampled = {k: random.sample(v, 10) for k, v in sampled.items()}

In [25]:
relation_map = {
    " causes ": "Causes",
    " includes ": "HasSubEvent",
    " can be hindered by ": "HinderedBy",
    " happens after ": "isAfter",
    " happens before ": "isBefore",
    " because ": "xReason"
}

In [26]:
new_dics = []
for k, v in sampled.items():
    for d in v:
        new_dics.append(
            {
                "relation": relation_map[k],
                "narrative": d["narrative"],
                "question": d["question"],
                "answer": d["answer"]
            }
        )

In [27]:
with open("../resources/atomic_2020_narratives_sampled.jsonl", "w") as f:
    for d in new_dics:
        f.write(json.dumps(d) + "\n")